PM Reporting Tool - POC Testing Script

This script demonstrates the PM reporting tool that aggregates employee updates 
and generates reports for stakeholders.

Features:
- Load mock employee standup updates
- Generate different types of reports (weekly, project status, blockers)
- Answer stakeholder questions using RAG approach
- Display team statistics and insights

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import our custom modules
from models import Update, AggregatedData
from mock_llm import MockLLM
from data_loader import DataLoader
from reporting_tool import PMReportingTool
from config import Config, ReportType
from free_llm_providers_old import create_llm

In [2]:
def print_section(title, char="=", length=60):
    """Print a formatted section header"""
    print(f"\n{char * length}")
    print(f"{title}")
    print(f"{char * length}")

def print_subsection(title, char="-", length=50):
    """Print a formatted subsection header"""
    print(f"\n{title}")
    print(f"{char * length}")

def setup_and_initialize():
    """Initialize the reporting tool and load mock data"""
    print_section("🚀 PM REPORTING TOOL - POC TESTING")
    
    # Choose LLM provider - you can change this to test different providers
    llm_provider = "ollama"  # Options: "mock", "ollama", "groq", "huggingface"
    
    print(f"🤖 Using LLM provider: {llm_provider}")
    
    # Initialize the LLM
    try:
        llm = create_llm(llm_provider)
        print(f"✅ {llm_provider} LLM initialized successfully")
    except Exception as e:
        print(f"⚠️  Failed to initialize {llm_provider} LLM: {e}")
        print("🔄 Falling back to mock LLM")
        llm = create_llm("mock")
    
    # Initialize the PM reporting tool with the chosen LLM
    tool = PMReportingTool(llm=llm)
    
    # Load mock data
    tool.load_mock_data()
    
    print("✅ All imports successful!")
    print("📊 PM Reporting Tool POC ready for testing")
    print(f"📝 Loaded {len(tool.updates)} initial updates")
    
    return tool

def explore_data(tool):
    """Explore and display information about the loaded data"""
    print_section("📊 DATA EXPLORATION")
    
    # Display summary statistics
    tool.display_summary_stats()
    
    # Show sample updates
    print_subsection("📋 Sample Team Updates:")
    
    for i, update in enumerate(tool.updates[:3]):
        print(f"\n{i+1}. {update.employee} ({update.role}) - {update.date}")
        print(f"   {update.update[:100]}...")
        print("-" * 50)

def generate_reports(tool):
    """Generate different types of reports"""
    print_section("📈 REPORT GENERATION")
    
    # Generate a weekly report
    print_subsection("📊 WEEKLY REPORT")
    weekly_report = tool.generate_report("weekly")
    print(weekly_report)
    
    # Generate a project status report
    print_subsection("🎯 PROJECT STATUS REPORT")
    status_report = tool.generate_report("project status")
    print(status_report)
    
    # Generate a blockers report
    print_subsection("🚫 BLOCKERS REPORT")
    blockers_report = tool.generate_report("blockers")
    print(blockers_report)

def stakeholder_qa(tool):
    """Demonstrate stakeholder Q&A functionality"""
    print_section("❓ STAKEHOLDER Q&A SESSION")
    
    # Answer stakeholder questions
    questions = Config.SAMPLE_QUESTIONS[:5]  # Use first 5 sample questions
    
    for i, question in enumerate(questions, 1):
        print(f"\n{i}. Q: {question}")
        answer = tool.answer_stakeholder_question(question)
        print(f"   A: {answer}")
        print("-" * 50)

def test_filtering_features(tool):
    """Test data filtering capabilities"""
    print_section("🔍 FILTERING FEATURES")
    
    # Test role-based filtering
    print_subsection("Updates by Role - Backend Developers:")
    backend_updates = tool.get_updates_by_role("Backend Developer")
    for update in backend_updates:
        print(f"• {update.employee}: {update.update[:80]}...")
    
    # Test employee-based filtering
    print_subsection("Updates by Employee - Sarah Chen:")
    sarah_updates = tool.get_updates_by_employee("Sarah Chen")
    for update in sarah_updates:
        print(f"• {update.date}: {update.update[:80]}...")

def test_additional_data(tool):
    """Test with additional mock data"""
    print_section("📈 TESTING WITH ADDITIONAL DATA")
    
    # Load additional mock data
    tool.load_additional_mock_data()
    tool.load_blocker_data()
    
    print(f"📝 Total updates after loading additional data: {len(tool.updates)}")
    
    # Generate updated reports
    print_subsection("📊 Updated Weekly Report:")
    updated_report = tool.generate_report("weekly")
    print(updated_report)
    
    # Show updated statistics
    print_subsection("📊 Updated Statistics:")
    stats = tool.get_summary_stats()
    print(f"Total Updates: {stats['total_updates']}")
    print(f"Team Members: {stats['team_members']}")
    print(f"Date Range: {stats['date_range']}")

def test_custom_questions(tool):
    """Test with custom stakeholder questions"""
    print_section("🎯 CUSTOM STAKEHOLDER QUESTIONS")
    
    custom_questions = [
        "What are the biggest risks for our Q4 launch?",
        "Which team members are working on the payment system?",
        "How many bugs were found in testing this week?",
        "What's the current status of the mobile app beta?",
        "Are there any resource constraints affecting the team?"
    ]
    
    for i, question in enumerate(custom_questions, 1):
        print(f"\n{i}. Q: {question}")
        answer = tool.answer_stakeholder_question(question)
        print(f"   A: {answer}")
        print("-" * 50)

def performance_insights(tool):
    """Display performance insights about the tool"""
    print_section("⚡ PERFORMANCE INSIGHTS")
    
    stats = tool.get_summary_stats()
    aggregated_data = tool.aggregate_updates()
    
    print(f"📊 Data Processing Stats:")
    print(f"• Total updates processed: {stats['total_updates']}")
    print(f"• Unique team members: {stats['team_members']}")
    print(f"• Role diversity: {stats['roles']} different roles")
    print(f"• Context length: {len(aggregated_data.get_context_string())} characters")
    
    print(f"\n🎯 Role Distribution:")
    for role, count in stats['updates_by_role'].items():
        percentage = (count / stats['total_updates']) * 100
        print(f"• {role}: {count} updates ({percentage:.1f}%)")


In [5]:
print("🤖 AVAILABLE FREE LLM OPTIONS:")
print("=" * 60)
print("1. Mock LLM (default) - No setup required")
print("2. Ollama - Completely free, runs locally")
print("   Setup: Install Ollama, run 'ollama pull llama2', start 'ollama serve'")
print("3. Groq - Free tier, very fast inference")
print("   Setup: Sign up at console.groq.com, set GROQ_API_KEY env var")
print("4. Hugging Face - Free tier available")
print("   Setup: Sign up at huggingface.co, set HUGGINGFACE_API_KEY env var")
print("\nTo use a different LLM, change the 'llm_provider' in setup_and_initialize()")
print("=" * 60)

🤖 AVAILABLE FREE LLM OPTIONS:
1. Mock LLM (default) - No setup required
2. Ollama - Completely free, runs locally
   Setup: Install Ollama, run 'ollama pull llama2', start 'ollama serve'
3. Groq - Free tier, very fast inference
   Setup: Sign up at console.groq.com, set GROQ_API_KEY env var
4. Hugging Face - Free tier available
   Setup: Sign up at huggingface.co, set HUGGINGFACE_API_KEY env var

To use a different LLM, change the 'llm_provider' in setup_and_initialize()


In [3]:
# Initialize and setup
tool = setup_and_initialize()


🚀 PM REPORTING TOOL - POC TESTING
🤖 Using LLM provider: ollama
✅ ollama LLM initialized successfully
✅ All imports successful!
📊 PM Reporting Tool POC ready for testing
📝 Loaded 9 initial updates


In [20]:
explore_data(tool)


📊 DATA EXPLORATION
=== TEAM UPDATES SUMMARY ===
Total Updates: 9
Team Members: 9
Roles Represented: 8
Date Range: 2024-07-08 to 2024-07-09

Updates by Role:
  Backend Developer: 2 update(s)
  Frontend Developer: 1 update(s)
  Product Manager: 1 update(s)
  DevOps Engineer: 1 update(s)
  QA Engineer: 1 update(s)
  Marketing Manager: 1 update(s)
  UX Designer: 1 update(s)
  Senior Developer: 1 update(s)

Updates by Employee:
  Sarah Chen: 1 update(s)
  Mike Rodriguez: 1 update(s)
  Lisa Wang: 1 update(s)
  David Kim: 1 update(s)
  Emma Thompson: 1 update(s)
  Alex Johnson: 1 update(s)
  Tom Wilson: 1 update(s)
  Jessica Brown: 1 update(s)
  Chris Lee: 1 update(s)


📋 Sample Team Updates:
--------------------------------------------------

1. Sarah Chen (Frontend Developer) - 2024-07-08
   Completed the new user authentication flow. The login/signup process is now much smoother and we've ...
--------------------------------------------------

2. Mike Rodriguez (Backend Developer) - 2024-

In [4]:
generate_reports(tool)


📈 REPORT GENERATION

📊 WEEKLY REPORT
--------------------------------------------------
Weekly Report for Stakeholders:

Key Achievements and Progress:

* Frontend Developer Sarah Chen completed the new user authentication flow and fixed mobile responsive issues on the dashboard. She is now working on integrating the design system components.
* Backend Developer Mike Rodriguez finished API endpoints for user management, optimized database queries by 40%, and started working on payment gateway integration.
* Product Manager Lisa Wang completed user research interviews and updated the roadmap based on findings. The mobile app beta is ready for testing with 50 users signed up.
* DevOps Engineer David Kim deployed a new monitoring dashboard, fixed load balancer issues, and is working on database migration planning for next month.
* QA Engineer Emma Thompson completed testing for the authentication system, created an automated test suite for the user dashboard, and started mobile app testi

In [12]:
stakeholder_qa(tool)


❓ STAKEHOLDER Q&A SESSION

1. Q: What's the status of the mobile app?
   A: 
**Mobile App Status Update:**

The mobile app is progressing well with 85% completion. Key highlights:
- UI/UX improvements have been implemented with positive user feedback
- Beta testing is currently in progress with 50 signed-up users
- Some UI inconsistencies identified during QA testing are being addressed
- Integration with the new authentication system is complete
- Next phase focuses on payment UI components and final testing

**Timeline:** Expected to complete beta testing by end of next week, with full release targeted for Q4.
        
--------------------------------------------------

2. Q: Are there any current blockers?
   A: 
## Current Blockers & Dependencies

### High Priority Blockers
1. **Third-party API Access**: Marketing team waiting for vendor approval (3 days)
2. **Database Migration**: Requires infrastructure team coordination (scheduled next week)
3. **Payment Gateway Integration**: 

In [22]:
test_filtering_features(tool)



🔍 FILTERING FEATURES

Updates by Role - Backend Developers:
--------------------------------------------------
• Mike Rodriguez: Finished the API endpoints for user management. All CRUD operations are working ...
• Tom Wilson: Implemented the new caching layer for the API. Response times improved by 30%. W...

Updates by Employee - Sarah Chen:
--------------------------------------------------
• 2024-07-08: Completed the new user authentication flow. The login/signup process is now much...


In [23]:
test_additional_data(tool)



📈 TESTING WITH ADDITIONAL DATA
📝 Total updates after loading additional data: 15

📊 Updated Weekly Report:
--------------------------------------------------
Error connecting to Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/generate. Make sure Ollama is running locally.

📊 Updated Statistics:
--------------------------------------------------
Total Updates: 15
Team Members: 9
Date Range: 2024-07-08 to 2024-07-10


In [15]:
test_custom_questions(tool)


🎯 CUSTOM STAKEHOLDER QUESTIONS

1. Q: What are the biggest risks for our Q4 launch?
   A: 
**Mobile App Status Update:**

The mobile app is progressing well with 85% completion. Key highlights:
- UI/UX improvements have been implemented with positive user feedback
- Beta testing is currently in progress with 50 signed-up users
- Some UI inconsistencies identified during QA testing are being addressed
- Integration with the new authentication system is complete
- Next phase focuses on payment UI components and final testing

**Timeline:** Expected to complete beta testing by end of next week, with full release targeted for Q4.
        
--------------------------------------------------

2. Q: Which team members are working on the payment system?
   A: 
**Mobile App Status Update:**

The mobile app is progressing well with 85% completion. Key highlights:
- UI/UX improvements have been implemented with positive user feedback
- Beta testing is currently in progress with 50 signed-up users

In [16]:
performance_insights(tool)


⚡ PERFORMANCE INSIGHTS
📊 Data Processing Stats:
• Total updates processed: 15
• Unique team members: 9
• Role diversity: 8 different roles
• Context length: 4527 characters

🎯 Role Distribution:
• Backend Developer: 3 updates (20.0%)
• Frontend Developer: 2 updates (13.3%)
• Product Manager: 2 updates (13.3%)
• DevOps Engineer: 2 updates (13.3%)
• QA Engineer: 2 updates (13.3%)
• Marketing Manager: 2 updates (13.3%)
• UX Designer: 1 updates (6.7%)
• Senior Developer: 1 updates (6.7%)


In [28]:
import requests

try:
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": "llama2", "prompt": "Say something smart!", "stream": False},
        timeout=30
    )
    print(response.json()["response"])
except Exception as e:
    print(f"Error: {e}")



"The only thing necessary for the triumph of evil is for good men to do nothing." - Edmund Burke.


In [ ]:
llm = OllamaLLM(model_name=model_name, base_url=base_url)